In [ ]:
%load_ext IPython.extensions.autoreload
%autoreload 2
%matplotlib ipympl
import sys
sys.path.append('C:\code\qdc2')
import matplotlib
matplotlib.use('qt')

In [ ]:
from matplotlib import pyplot as plt

from qdc.diffuser.diffuser_result import DiffuserResult
from qdc.diffuser.diffuser_sim import DiffuserSimulation
import numpy as np 

show_all = False

ress = []
for i in range(30):
    N2pi = 40
    rms_height = 2*np.pi * N2pi
    N_wl = 41 
    diffuser_angle = 3 * 2*np.pi/360 
    with_dispersion = True
    
    sim = DiffuserSimulation(
        Nx=1024, Ny=1024,
        Lx=8e-3, Ly=8e-3,
        wl0=808e-9,
        Dwl=300e-9,
        N_wl=N_wl,  
        waist=40e-6,
        focal_length=200e-3,
        init_off_axis=0.,
        diffuser_angle=diffuser_angle,
        rms_height=rms_height, 
        with_dispersion=with_dispersion
    )
    
    res = sim.run_SPDC_simulation(populate_res=True)
    res = sim.run_classical_simulation(populate_res=True)
    ress.append(res)
# res.show()
# plt.show(block=True)

In [ ]:
SPDC_PCCs = np.array([res.SPDC_PCCs for res in ress]).mean(axis=0)  
classical_PCCs = np.array([res.classical_PCCs for res in ress]).mean(axis=0)

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(ress[0].SPDC_delta_lambdas*1e9, SPDC_PCCs, '-', label='SPDC')
ax.plot(ress[0].classical_delta_lambdas*1e9, classical_PCCs, '-', label='Classical', color='#8c564b')
ax.set_xlabel('$\Delta\lambda$ [nm]', fontsize=20)
ax.set_ylabel('PCC', fontsize=20)
ax.legend(loc='center', fontsize=24)
ax.tick_params(axis='x', labelsize=22)  # tick label size
ax.tick_params(axis='y', labelsize=22)  # tick label size
ax.set_xticks([0, 100, 200, 300])

fig.show()
plt.show(block=True)
fig.savefig(r'G:\My Drive\Projects\Dispersion Cancelation\Paper\Partial Figures\diffuser_PCCs_inset.png')
# fig.savefig(r'G:\My Drive\Projects\Dispersion Cancelation\Paper\Partial Figures\diffuser_PCCs.png')

In [ ]:
res = ress[-1]
saveto_path = r'G:\My Drive\Projects\Dispersion Cancelation\Paper\Partial Figures\diffuser_incoherent_sum_classical.png'
res.classical_incoherent_sum /= res.classical_incoherent_sum.max()
res.show_incoherent_sum_classical(clean=True, title='', add_square=True, saveto_path=saveto_path)
saveto_path = r'G:\My Drive\Projects\Dispersion Cancelation\Paper\Partial Figures\diffuser_incoherent_sum_SPDC.png'
res.SPDC_incoherent_sum /= res.SPDC_incoherent_sum.max()
res.show_incoherent_sum_SPDC(clean=True, title='', add_square=True, saveto_path=saveto_path)
# res.show_PCCs()
plt.show(block=True)

In [ ]:
saveto_path = r'G:\My Drive\Projects\Dispersion Cancelation\Paper\Partial Figures\diffuser1_classical_1.png'
# saveto_path = None 
res.classical_incoherent_sum /= res.classical_incoherent_sum.max()
res.show_incoherent_sum_classical(clean=True, title='', add_square=False, saveto_path=saveto_path)
plt.show(block=True)

In [ ]:
# Make movies
import matplotlib.animation as animation
from qdc.misc import colorize
colorize = lambda x: np.abs(x)**2
import matplotlib as mpl
path = r"C:\Users\owner\AppData\Roaming\Python\Python310\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe"
mpl.rcParams['animation.ffmpeg_path'] = path


def make_movie(output_fields, dl, save_path, is_SPDC=True):
    fps = 4
    fig, ax = plt.subplots(constrained_layout=True)

    def animate(frame_idx):
        ax.clear()

        field = output_fields[frame_idx]

        ax.imshow(colorize(field), cmap='viridis')
        text = f'$\lambda_2-\\lambda_0={dl[frame_idx]*1e3:.1f} nm$'
        ax.text(0.5, 0.97, text, color='white', fontsize=18, fontweight='bold',
                va='top', ha='center', transform=ax.transAxes)

        return ax

    anim = animation.FuncAnimation(fig, animate, frames=len(output_fields)-1,
                                    interval=1000//fps, blit=False, repeat=False)

    anim.save(save_path, writer='ffmpeg', fps=fps)
    plt.close(fig)

base_path = r"G:\My Drive\Projects\Dispersion Cancelation\Paper\Movies\Diffuser"
fields_SPDC = np.array([res.SPDC_fields[i].E for i in range(len(res.SPDC_fields))])[::-1]
dls_SPDC = (res.wavelengths - res.wl0)[::-1]*1e6
fields_classical = np.array([res.classical_fields[i].E for i in range(len(res.classical_fields))])
dls_classical = res.classical_delta_lambdas*1e6
make_movie(fields_SPDC, dls_SPDC, f"{base_path}_SPDC.mp4", is_SPDC=True)
make_movie(fields_classical, dls_classical, f"{base_path}_classical.mp4", is_SPDC=False)
